# Practicum 2: Advection equation

This practicum looks at various effects of space and time discretization by solving the advection equation

$$\frac{\partial \psi}{\partial t}+c\frac{\partial \psi}{\partial x}=0$$


## Preparations

We start with loading the numpy and matplotlib libraries, and by defining a function to show animations.

In [1]:
## Load numpy and matplotlib libraries
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.ioff()
plt.rcParams["animation.html"] = "jshtml"
plt.rcParams['figure.dpi'] = 100  # reduce to generate figures faster (but smaller)

# function for animating results
def create_animation(phi,frames=None,labels=None,ylim=None):
    
    # create figure with axes
    fig,ax=plt.subplots()
    
    if len(phi.shape)==2:
        phi=phi.reshape([phi.shape[0],phi.shape[1],1])
    
    # number of experiments
    nExp=phi.shape[2]

    # determine frames to plot
    if frames is None:
        # number of timesteps
        nt=phi.shape[0]-1
        frames=np.arange(0,nt+1)
    else:
        nt=frames[-1]
    
    
    # set y limits
    if ylim is None:
        ymin=np.min(phi[0,:,:])
        ymax=np.max(phi[0,:,:])
        ylim=[1.2*ymin-0.2*ymax,1.2*ymax-0.2*ymin]
    
    # labels and legend   
    showlegend=True
    if labels is None:
        showlegend=False
        labels=['exp %i'%jExp for jExp in range(nExp)]
        
    # create lines for numerical solution
    ll_n=[ax.plot(x,phi[frames[0],:,jExp],label=labels[jExp])[0] for jExp in range(nExp)]
    
    # set ylim
    ax.set_ylim(ylim)
    
    # add legend
    if showlegend:
        plt.legend()
    
    # add title
    tt=plt.title('')
    

    def animate(iframe):
        for jExp in range(nExp):
            ll_n[jExp].set_ydata(phi[iframe,:,jExp])
        tt.set_text('timestep %i/%i'%(iframe,nt))

    anim=animation.FuncAnimation(fig, animate, frames=frames,cache_frame_data=False,blit=True)
    display(anim)


## 1. The upstream scheme

The upstream scheme discretizes a differential equation in time as

$$\frac{\phi^{n+1}_j-\phi^n_j}{\Delta t}+c\frac{\phi^{n}_j-\phi^{n}_{j-1}}{\Delta x}=0,$$

where $n$ denotes the timestep ($t=n\Delta t$), and $j$ denotes the gridpoint ($x=j\Delta x$). From this equation, the solution at the next timestep is solved as

$$\phi_j^{n+1}=\phi_j^n-\mu(\phi_j^n-\phi_{j-1}^n)$$.

with $\mu=\frac{c\Delta t}{\Delta x}$ is called the Courant number.

**Note about periodic boundary conditions**:

Periodic boundary conditions are assumed. This means that an index $-1$ (point left of point with index $0$) gets replaced by an index $nx-1$ (last point), and that an index $n$ gets replaced by $0$. This is conveniently coded in python as

```
jL=(j-1)%nx    # index of point left of j
jR=(j+1)%nx    # index of point right of j
```

The operator `%` is the modulus operator (remainder after division). It can be verified easily that for `j=0`, `jL=nx-1`, and for `j=nx-1`, `jR=0`.


In [2]:
# parameters
c=0.8

dt=1.0
nt=200

nx=128
dx=1.0

# courant number
mu=c*dt/dx
print('Courant number: mu = ',mu)

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# exact solution
def exact_solution(x):
    if True:
        # Gaussian bell
        xs=(2*x/(nx*dx))%2-1   # scaled between -1 and 1
        psi=np.exp(-10*xs**2)
    elif True:
        # harmonic function with wavenumber k
        k=2
        psi=np.cos(k*(2*np.pi/(nx*dx))*x)
    return(psi)

# allocate solution
nExp=2   # 2 experiments: exact and numerical
phi=np.zeros((nt+1,nx,nExp))
for jExp in range(nExp):
    phi[0,:,jExp]=exact_solution(x)

# indices
j=np.arange(nx)    # 0,1,...,nx-1
jL=(j-1)%nx        # nx-1,0,1,...,nx-2
jR=(j+1)%nx        # 1,2,...,nx-1,0
    
# evolution in time
for jt in range(nt):
    phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))        # exact solution
    phi[jt+1,j,1]=(1-mu)*phi[jt,j,1]+mu*phi[jt,jL,1]   # upstream scheme

# animate results
create_animation(phi,labels=['exact','upstream'],ylim=[-3,3],frames=np.arange(0,nt,nt//20))

Courant number: mu =  0.8


**Exercise:** Modify the values of `dt`, `dx` and `c`. Verify that the upstream scheme has the following stability condition:

$$0\leq\mu\leq1$$.

**Exercise:** Verify if the damping is equally strong for harmonic functions with different wavelengths (change `k` in the exact solution).

**Question:** Why is the damping/amplification usually stronger for short waves than for long waves? 

## Forward scheme, centered spatial finite differences

Let's replace the 1st order decentered spatial finite differences from the upstream scheme with more accurate 2nd order centered spatial finite differences:

$$\frac{\phi^{n+1}_j-\phi^n_j}{\Delta t}+c\frac{\phi^{n}_{j+1}-\phi^{n}_{j-1}}{2\Delta x}=0,$$

The next timestep's solution is given as

$$\phi^{n+1}_j=\phi^n_j-\frac{c\Delta t}{2\Delta x}(\phi^{n}_{j+1}-\phi^{n}_{j-1})$$

**Exercise**: Implement this scheme, starting with a copy of the upstream scheme. Verify the stability. 


In [ ]:
# parameters
c=0.5

dt=1.0
nt=200

nx=128
dx=1.0

# courant number
mu=c*dt/dx
print('Courant number: mu = ',mu)

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# exact solution
def exact_solution(x):
    if True:
        # Gaussian bell
        xs=(2*x/(nx*dx))%2-1   # scaled between -1 and 1
        psi=np.exp(-10*xs**2)
    elif True:
        # harmonic function with wavenumber k
        k=2
        psi=np.cos(k*(2*np.pi/(nx*dx))*x)
    return(psi)

# allocate solution
nExp=2   # 2 experiments: exact and numerical
phi=np.zeros((nt+1,nx,nExp))
for jExp in range(nExp):
    phi[0,:,jExp]=exact_solution(x)

# indices
j=np.arange(nx)    # 0,1,...,nx-1
jL=(j-1)%nx        # nx-1,0,1,...,nx-2
jR=(j+1)%nx        # 1,2,...,nx-1,0
    
# evolution in time
for jt in range(nt):
    phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))        # exact solution
    phi[jt+1,j,1]=phi[jt,j,1]-c*dt/(2*dx)*(phi[jt,jR,1]-phi[jt,jL,1])   # forward+centered scheme

# animate results
create_animation(phi,labels=['exact','forward centered'],ylim=[-3,3],frames=np.arange(0,nt,nt//20))

You should find that this scheme is unconditionally unstable!

**Question**: We moved to a more accurate calculation of the spatial derivatives. How is it possible that this leads to worse results?

## Leapfrog scheme, centered spatial finite differences

Let's move to a time scheme that's more stable than the forward scheme: the 3-timelevel leapfrog scheme:

$$\frac{\phi^{n+1}_j-\phi^{n-1}_j}{2\Delta t}+c\frac{\phi^{n}_{j+1}-\phi^{n}_{j-1}}{2\Delta x}=0,$$

The next timestep's solution is given as

$$\phi^{n+1}_j=\phi^{n-1}_j-\frac{c\Delta t}{\Delta x}(\phi^{n}_{j+1}-\phi^{n}_{j-1})$$

**Exercises**:
1. Implement this scheme, starting with a copy of the previous scheme. Note that during the first timestep, you cannot use the leapfrog scheme since there is no previous timestep available yet. You can use the forward scheme for the first timestep.
2. Verify the stability as a function of the Courant number $\mu=\frac{c\Delta t}{\Delta x}$.
3. The leapfrog scheme is accelerating, centered finite differences are decelerating. Verify if the combined scheme is accelerating or decelerating.
4. Check the wavenumber dependency of the phase speed by considering very short waves (high `k` in the exact solution).
5. Try to illustrate the negative group velocity for the short waves by using a spike as initial condition.



In [ ]:
# parameters
c=0.5

dt=1.0
nt=200

nx=128
dx=1.0

# courant number
mu=c*dt/dx
print('Courant number: mu = ',mu)

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# exact solution
def exact_solution(x):
    if True:
        # Gaussian bell
        xs=(2*x/(nx*dx))%2-1   # scaled between -1 and 1
        psi=np.exp(-10*xs**2)
    elif True:
        # harmonic function with wavenumber k
        k=2
        psi=np.cos(k*(2*np.pi/(nx*dx))*x)
    return(psi)

# allocate solution
nExp=2   # 2 experiments: exact and numerical
phi=np.zeros((nt+1,nx,nExp))
for jExp in range(nExp):
    phi[0,:,jExp]=exact_solution(x)

# indices
j=np.arange(nx)    # 0,1,...,nx-1
jL=(j-1)%nx        # nx-1,0,1,...,nx-2
jR=(j+1)%nx        # 1,2,...,nx-1,0
    
# evolution in time
for jt in range(nt):
    phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))        # exact solution
    phi[jt+1,j,1]=phi[jt-1,j,1]-c*dt/dx*(phi[jt,jR,1]-phi[jt,jL,1])   # leapfrog+centered scheme

# animate results
create_animation(phi,labels=['exact','leapfrog centered'],ylim=[-3,3],frames=np.arange(0,nt,nt//20))

## Implicit schemes with spatial finite differences

Implicit time schemes such as the backward scheme or the trapezium scheme are much more stable than explicit schemes, so they allow to take a very large timestep.

**Exercise**: Think about implementing the trapezium scheme:

$$\frac{\phi^{n+1}_j-\phi^{n}_j}{\Delta t}+\frac{c}{2}\left(
    \frac{\phi^{n}_{j+1}-\phi^{n}_{j-1}}{2\Delta x}+\frac{\phi^{n+1}_{j+1}-\phi^{n+1}_{j-1}}{2\Delta x}\right)=0$$

Why would this be quite difficult?


In [ ]:
# put your attempt here

## Implicit spectral schemes

When formulating the trapezium scheme (for a linear problem) in spectral space, the different waves become *decoupled*. This makes this combination especially interesting. For the advection equation, we have

$$\phi(t,x)=\sum_{k=-N}^{N} \alpha_k(t) e^{ikx}$$ 

where $\alpha_k(t)$ are the *spectral coefficients*. The time evolution of each spectral coefficient then becomes

$$\frac{\partial \alpha_k}{\partial t}+cik\alpha_k=0.$$

which is an ordinary differential equation instead of a partial differential equation. Applying the trapezium scheme now becomes much easier:

$$\frac{\alpha_k^{n+1}-\alpha_k^n}{\Delta t}+\frac{ick}{2}\left(\alpha_k^n+\alpha_k^{n+1}\right)=0,$$

from which $\alpha_k^{n+1}$ can be solved as

$$\alpha_k^{n+1}=\frac{1-ick\Delta t/2}{1+ick\Delta t/2}\alpha_k^n.$$

The advection equation solved with a trapezium+spectral scheme is given below:

In [ ]:
# parameters
c=0.5

dt=1.0
nt=200

nx=128
dx=1.0

# courant number
mu=c*dt/dx
print('Courant number: mu = ',mu)

# define coordinates
x=np.arange(nx)*dx
t=np.arange(nt)*dt

# exact solution
def exact_solution(x):
    if True:
        # Gaussian bell
        xs=(2*x/(nx*dx))%2-1   # scaled between -1 and 1
        psi=np.exp(-10*xs**2)
    else:
        # harmonic function with wavenumber k
        k=2
        psi=np.cos(k*(2*np.pi/(nx*dx))*x)
    return(psi)

# allocate solution
nExp=2   # 2 experiments: exact and numerical
phi=np.zeros((nt+1,nx,nExp))
for jExp in range(nExp):
    phi[0,:,jExp]=exact_solution(x)

# wavenumbers: the convention is to order them as [0,1,...,nx/2-1,-nx/2,...,-1]
k=(np.arange(nx)+nx/2)%nx-nx/2   # believe it or not, this yields the above list

# since the domain isn't 2*pi, the wavenumbers are scaled by 2*pi/(nx*dx)
k=(2*np.pi)/(nx*dx)*k
    
# evolution in time
for jt in range(nt):
    phi[jt+1,:,0]=exact_solution(x-c*dt*(jt+1))        # exact solution
    
    # fast fourier transform of current solution
    alpha=np.fft.fft(phi[jt,:,1])
    
    # evolution of alpha with the trapezium scheme
    alpha=(1-1j*c*k*dt/2)/(1+1j*c*k*dt/2)*alpha
    
    # next timestep's solution is inverse fourier transform; taking real part to remove imaginary component due to round-off errors
    phi[jt+1,:,1]=np.real(np.fft.ifft(alpha))

# animate results
create_animation(phi,labels=['exact','trapezium+spectral'],ylim=[-3,3],frames=np.arange(0,nt,nt//20))

**Exercises**:
1. Try to make this scheme unstable by playing with `dx`, `dt` and/or `c`.
2. Explain your observations by considering the discrete dispersion relation for this scheme:
$$\omega=\frac{2}{\Delta t}\text{arctan}\left(\frac{ck\Delta t}{2}\right)$$.
